In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# CHSH-Ungliichig

*Nutzigschätzig: Zwei Minute uf eme Heron r2-Prozässor (HIIWIS: Das isch nume e Schätzig. Eui Laufzyt chönnt variiere.)*
## Hingergrund
I däm Tutorial füehred mer es Experiment uf eme Quantecomputer uus, zum d'Verletzig vo de CHSH-Ungliichig mit em Estimator-Primitiv z'zeige.

D'CHSH-Ungliichig, benannt nach de Autoore Clauser, Horne, Shimony und Holt, wird bruucht, zum Bells Theorem (1969) experimentell z'bewiise. Das Theorem sait uus, dass lokali Theoriie mit versteckte Variable nöd alli Konsequenze vo de Verschränkig i de Quantemechanik chönd erkläre. D'Verletzig vo de CHSH-Ungliichig wird bruucht, zum z'zeige, dass d'Quantemechanik mit lokale Theoriie mit versteckte Variable nöd vereinbar isch. Das isch es wichtigs Experiment für s'Verständnis vo de Grundlage vo de Quantemechanik.

De Nobelpriis für Physik 2022 isch a Alain Aspect, John Clauser und Anton Zeilinger vergää worde, under anderem für ihri Pionierarbeit i de Quanteinformationswüsseschaft und bsunders für ihri Experimänt mit verschränkte Photone, wo d'Verletzig vo de Bellsche Ungliichige demonstriert händ.
## Aaforderige
Bevor dir mit däm Tutorial aafanged, stellet sicher, dass dir Folgendes installiert händ:

* Qiskit SDK v1.0 oder neuer, mit [visualization](https://docs.quantum.ibm.com/api/qiskit/visualization)-Understützig
* Qiskit Runtime (`pip install qiskit-ibm-runtime`) v0.22 oder neuer
## Iirichtig

In [1]:
# General
import numpy as np

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Qiskit Runtime imports
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator

# Plotting routines
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

## Schritt 1: Klassischi Iigabe uf es Quanteproblem abbildä
Für das Experiment erstelled mer es verschränkts Paar, wo mer jedes Qubit uf zwei verschideni Base mässed. Mer bezäichned d'Base für s'erscht Qubit mit $A$ und $a$ und d'Base für s'zweit Qubit mit $B$ und $b$. Das erlaubt öis, d'CHSH-Grössi $S_1$ z'berächne:

$$
S_1 = A(B-b) + a(B+b).
$$

Jedi Observable isch entweder $+1$ oder $-1$. Klar isch, dass äine vo de Terme $B\pm b$ gliich $0$ sii muess und de ander $\pm 2$ sii muess. Darum isch $S_1 = \pm 2$. De Durschnittswärt vo $S_1$ muess d'Ungliichig erfülle:

$$
|\langle S_1 \rangle|\leq 2.
$$

Wenn mer $S_1$ i Bezug uf $A$, $a$, $B$ und $b$ uswigglet, chömed mer uf:

$$
|\langle S_1 \rangle| = |\langle AB \rangle - \langle Ab \rangle + \langle aB \rangle + \langle ab \rangle| \leq 2
$$

Dir chönnt e wyteri CHSH-Grössi $S_2$ definiere:

$$
S_2 = A(B+b) - a(B-b),
$$

Das füehrt zu enere wyytere Ungliichig:

$$
|\langle S_2 \rangle| = |\langle AB \rangle + \langle Ab \rangle - \langle aB \rangle + \langle ab \rangle| \leq 2
$$

Wenn d'Quantemechanik dur lokali Theoriie mit versteckte Variable cha beschribe wärde, müend d'vorige Ungliichige wohr sii. Abelewi däm Tutorial aber gzäigt wird, chönd die Ungliichige uf eme Quantecomputer verletzt wärde. Darum isch d'Quantemechanik nöd mit lokale Theoriie mit versteckte Variable vereinbar.
Falls dir meh Theoriie lerned wänd, luäged euch [Entanglement in Action](/learning/courses/basics-of-quantum-information/entanglement-in-action/chsh-game) mit em John Watrous aa.
Mer erstelled es verschränkts Paar zwüschet zwei Qubits i eme Quantecomputer, indem mer de Bell-Zuestand $|\Phi^+\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}}$ erzüüged. Mit em Estimator-Primitiv chönnt dir diräkt di nötige Erwartungswärt ($\langle AB \rangle, \langle Ab \rangle, \langle aB \rangle$ und $\langle ab \rangle$) überchoo, zum d'Erwartungswärt vo de beide CHSH-Grössene $\langle S_1\rangle$ und $\langle S_2\rangle$ z'berächne. Vor de Iifüehrig vom Estimator-Primitiv hätt dir d'Erwartungswärt us de Messresultat müesse konstruiere.

Mer mässed s'zweit Qubit i de $Z$- und $X$-Base. S'erscht Qubit wird au i orthogonale Base gmässe, aber mit eme Winkel bezüglich vom zweite Qubit, wo mer zwüschet $0$ und $2\pi$ variiere wärded. Wie dir gsehnd, macht s'Estimator-Primitiv s'Usfüehre vo parametrisierte Schaltchreis sehr eifach. Anstatt e Reihe vo CHSH-Schaltchreis z'erstelle, müend dir nume *eine* CHSH-Schaltchreis mit eme Parameter erstelle, wo de Mässwinkel aagit, und e Reihe vo Phasewärt für de Parameter.

Schlussändlich wärded mer d'Resultat analysiere und gäge de Mässwinkel ufträge. Dir wärded gseh, dass es für en bestimmte Bereich vo Mässwinkel d'Erwartungswärt vo de CHSH-Grössene $|\langle S_1\rangle| > 2$ oder $|\langle S_2\rangle| > 2$ sind, was d'Verletzig vo de CHSH-Ungliichig demonstriert.

In [2]:
# To run on hardware, select the backend with the fewest number of jobs in the queue
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
backend.name

'ibm_kingston'

### Create a parameterized CHSH circuit

First, we write the circuit with the parameter $\theta$, which we call `theta`. The [`Estimator` primitive](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) can enormously simplify circuit building and output analysis by directly providing expectation values of observables. Many problems of interest, especially for near-term applications on noisy systems, can be formulated in terms of expectation values. `Estimator` (V2) primitive can automatically change measurement basis based on the supplied observable.

In [3]:
theta = Parameter("$\\theta$")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)
chsh_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif" alt="Output of the previous code cell" />

### E parametrisierte CHSH-Schaltchreis erstelle
Zerscht schriibemer de Schaltchreis mit em Parameter $\theta$, wo mer `theta` nänned. S'[`Estimator`-Primitiv](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) cha de Schaltchreisbou und d'Usgabenaanalyse riisig veryfache, indem's diräkt Erwartungswärt vo Observable lieferet. Vieli intressanti Problem, bsunders für churzfristi Aawändige uf vrauschte Systeem, chönd i Form vo Erwartungswärt formuliert wärde. S'`Estimator` (V2)-Primitiv cha automatisch d'Mässbasis basierend uf de bereitgstellte Observable ändere.

In [4]:
number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
# Phases need to be expressed as list of lists in order to work
individual_phases = [[ph] for ph in phases]

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif)

### E Liste vo Phasewärt erstelle, wo schpöter zuegwise wärded
Nachdem dir de parametrisiert CHSH-Schaltchreis erstellt händ, erstellt dir e Liste vo Phasewärt, wo em Schaltchreis im nächste Schritt zuegwise wärded. Dir chönnt de folgende Code bruuche, zum e Liste vo 21 Phasewärt im Bereich vo $0$ bis $2 \pi$ mit gliichem Abstand z'erstelle, also $0$, $0.1 \pi$, $0.2 \pi$, ..., $1.9 \pi$, $2 \pi$.

In [5]:
# <CHSH1> = <AB> - <Ab> + <aB> + <ab> -> <ZZ> - <ZX> + <XZ> + <XX>
observable1 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", -1), ("XZ", 1), ("XX", 1)]
)

# <CHSH2> = <AB> + <Ab> - <aB> + <ab> -> <ZZ> + <ZX> - <XZ> + <XX>
observable2 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", 1), ("XZ", -1), ("XX", 1)]
)

### Observable
Jetzt bruuched mer Observable, us dänä mer d'Erwartungswärt chönd berächne. I öisem Fall betrachted mer orthogonali Base für jedes Qubit, wobii d'parametrisiert $Y$-Rotation für s'erscht Qubit d'Mässbasis beinaah kontinuierlich i Bezug uf d'Basis vom zweite Qubit variiert. Mer wähled darum d'Observable $ZZ$, $ZX$, $XZ$ und $XX$.

In [6]:
target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

chsh_isa_circuit = pm.run(chsh_circuit)
chsh_isa_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif" alt="Output of the previous code cell" />

## Schritt 2: Problem für d'Usfüehrig uf Quantehardware optimiere

Zum d'Gsamtusfüehrigszyt vom Job z'reduziere, akzeptiere V2-Primitiv nume Schaltchreis und Observable, wo de vom Ziilsystem understützte Aawysige und de Konnektivität entspräched (bezäichnet als Instruction Set Architecture (ISA)-Schaltchreis und -Observable).

### ISA-Schaltchreis

In [7]:
isa_observable1 = observable1.apply_layout(layout=chsh_isa_circuit.layout)
isa_observable2 = observable2.apply_layout(layout=chsh_isa_circuit.layout)

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif)

### ISA-Observable

Ebenso müend mer d'Observable transformiere, zum si backend-kompatibel z'mache, bevor mer Jobs mit [`Runtime Estimator V2`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2#run) usfüehred. Mer chönd d'Transformation mit de `apply_layout`-Methode vom `SparsePauliOp`-Objäkt durchfüehre.

In [8]:
# To run on a local simulator:
# Use the StatevectorEstimator from qiskit.primitives instead.

estimator = Estimator(mode=backend)

pub = (
    chsh_isa_circuit,  # ISA circuit
    [[isa_observable1], [isa_observable2]],  # ISA Observables
    individual_phases,  # Parameter values
)

job_result = estimator.run(pubs=[pub]).result()

## Schritt 3: Usfüehre mit Qiskit-Primitiv
Zum s'gsamti Experiment i eim einzige Uuruef vom [`Estimator`](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) uuszfüehre.
Mer chönd es [Qiskit Runtime `Estimator`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2)-Primitiv erstelle, zum öisi Erwartungswärt z'berächne. D'`EstimatorV2.run()`-Methode nimmt es Iterable vo `primitive unified blocs (PUBs)`. Jedes PUB isch es Iterable im Format `(circuit, observables, parameter_values: Optional, precision: Optional)`.

In [9]:
chsh1_est = job_result[0].data.evs[0]
chsh2_est = job_result[0].data.evs[1]

In [10]:
fig, ax = plt.subplots(figsize=(10, 6))

# results from hardware
ax.plot(phases / np.pi, chsh1_est, "o-", label="CHSH1", zorder=3)
ax.plot(phases / np.pi, chsh2_est, "o-", label="CHSH2", zorder=3)

# classical bound +-2
ax.axhline(y=2, color="0.9", linestyle="--")
ax.axhline(y=-2, color="0.9", linestyle="--")

# quantum bound, +-2√2
ax.axhline(y=np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.axhline(y=-np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.fill_between(phases / np.pi, 2, 2 * np.sqrt(2), color="0.6", alpha=0.7)
ax.fill_between(phases / np.pi, -2, -2 * np.sqrt(2), color="0.6", alpha=0.7)

# set x tick labels to the unit of pi
ax.xaxis.set_major_formatter(tck.FormatStrFormatter("%g $\\pi$"))
ax.xaxis.set_major_locator(tck.MultipleLocator(base=0.5))

# set labels, and legend
plt.xlabel("Theta")
plt.ylabel("CHSH witness")
plt.legend()
plt.show()

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/f6267448-0.avif" alt="Output of the previous code cell" />

In the figure, the lines and gray areas delimit the bounds; the outer-most (dash-dotted) lines delimit the quantum-bounds ($\pm 2$), whereas the inner (dashed) lines delimit the classical bounds ($\pm 2\sqrt{2}$). You can see that there are regions where the CHSH witness quantities exceeds the classical bounds. Congratulations! You have successfully demonstrated the violation of CHSH inequality in a real quantum system!

## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_3xxAgm1SF1wGp9k)